In [1]:
import pandas as pd
from thunderpack import ThunderReader
from openpyxl import load_workbook

In [3]:
#get list of patients with missing discharge date from excel file
#include patient ID and encounter ID in the list
df = pd.read_excel('/home/cdac-c-15/Desktop/cardiac_filtered_orginal.xlsx')

missingDataPIDs = df[df['HospitalDischargeDTS'].isna()]['BDSPPatientID'].tolist()
missingDataEIDs = df[df['HospitalDischargeDTS'].isna()]['BDSPEncounterID'].tolist()
dfNoDischargeDate = pd.DataFrame({
    'PatientID': missingDataPIDs,
    'EncounterID': missingDataEIDs
})

dfNoDischargeDate.head()

,PatientID,EncounterID
0,111249291,1565487169
1,111249291,1565487169
2,111249291,1565487169
3,111249291,1565487169
4,111249291,1565487169


In [ ]:
contains_pair = ((dfNoDischargeDate['PatientID'] == 113141131) & (dfNoDischargeDate['EncounterID'] == 418904088)).any()
print(f"{contains_pair}")

#checking that the list contains the correct pairs. true for 2 different IDS
#bad ids check? passed for both lists
#good ids check? passed for both lists


False


In [4]:
noDischPID = dfNoDischargeDate['PatientID']
noDischPID = dfNoDischargeDate.drop_duplicates()
print(len(noDischPID))
noDischPID.head()

491


,PatientID,EncounterID
0,111249291,1565487169
5,111317668,1602997059
14,111317668,1602996931
20,111504344,1625646484
29,111701935,1626494441


In [ ]:
# could use adt (transfer notes)
# 

reader = ThunderReader('/media/cdac-c-15/Thunderpack/Dropbox/zz_EHR_Thunderpacks/MGB/thunderpack_demographics_MGB')
dfs = []
for i in range(1, (len(list(reader.keys())) + 1)):
    df = reader[f'ICD_partition_{i}']
    dfc = df[df['BDSPEncounterID'].isin(noDischPID['EncounterID'])]
    dfs.append(df)


In [ ]:
filtered_icd_df = pd.concat(dfs, axis=0, ignore_index=True)     #puts data into rows and columns
print(len(df))      #print the number of notes found with those ICD codes
filtered_icd_df.head()      #print the first few lines of the filtered list of notes

1178760


,BDSPPatientID,DateOfBirth,DateOfDeath,DateOfDeathMARegistryData,PatientRace,EthnicGroupDSC,MaritalStatusDSC,ReligionDSC,LanguageDSC,VeteranStatusDSC,...,PrimaryCauseOfDeathDSC,UNOSPrimaryCauseOfDeathTXT,FirstContributoryCauseOfDeathDSC,UNOSContributoryCauseOfDeath01TXT,SecondContributoryCauseOfDeathDSC,UNOSContributoryCauseOfDeath02TXT,EducationLevelDSC,GenderIdentityDSC,SexAssignedAtBirthDSC,BDSPLastModifiedDTS
0,119106063,1963-06-28,None,None,Black or African American,Unavailable,Life Partner,Not Affiliated,English,"No, Never Served or Is Currently Active",...,None,None,None,None,None,NaN,Other,None,None,2022-07-17 17:57:14.1830000
1,114886961,2000-03-12,None,None,Other,Hispanic,Single,Not Affiliated,Spanish,"No, Never Served or Is Currently Active",...,None,None,None,None,None,NaN,Graduated - High School,Male,Male,2022-07-17 17:57:14.1830000
2,113203278,1971-07-09,None,None,Black or African American,Not Hispanic,Married/Civil Union,Muslim,English,"No, Never Served or Is Currently Active",...,None,None,None,None,None,NaN,Graduated - College,Male,Male,2022-07-17 17:57:14.1830000
3,115097977,1980-03-16,None,None,White,Not Hispanic,Single,Unavailable,English,Unavailable,...,None,None,None,None,None,NaN,Unavailable,Male,Male,2022-07-17 17:57:14.1830000
4,113470261,1991-01-27,None,None,White,Not Hispanic,Single,Roman Catholic,English,"No, Never Served or Is Currently Active",...,None,None,None,None,None,NaN,Graduated - High School,Female,Female,2022-07-17 17:57:14.1830000


In [ ]:
#merge dfs to make dfs contain missing data
dfNoDischargeDate = pd.merge(filtered_icd_df, dfNoDischargeDate, left_on=['BDSPPatientID', 'BDSPEncounterID'], right_on=['PatientID', 'EncounterID'], how='inner')
dfNoDischargeDate.head()

NameError: name 'filtered_icd_df' is not defined

In [ ]:
excel_df = pd.read_excel('/home/cdac-c-15/Desktop/cardiac_filtered_orginal.xlsx')

In [ ]:
#merge excel sheet with existing data
# Assuming 'PatientID' and 'EncounterID' are columns in both DataFrames
# and we want to update the 'Data_Column' in the Excel DataFrame

for index, row in dfNoDischargeDate.iterrows():
    id_value = row['PatientID']
    encounter_id_value = row['EncounterID']
    
    # Find the index in the original Excel DataFrame
    match_index = excel_df[(excel_df['PatientID'] == id_value) & (excel_df['EncounterID'] == encounter_id_value)].index
    
    # Update the Excel DataFrame with the new data
    if not match_index.empty:
        excel_df.loc[match_index, 'DischargeDate'] = row['NewDischargeDate']  # Replace 'Data_Column' with the actual column name to update

print("Updated Excel DataFrame:")
print(excel_df)